## Batch withdrawals tests

In [1]:
from docs.scenarios.setup import *

accounts = setup()
web3, etherlink_account, tezos_account = accounts
_, ticketer, erc20, token_bridge_helper, _ = load_contracts(*accounts)

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `100.377680 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `1.691780386 ꜩ`


### Deploy BulkWithdrawal:
- TODO: rename to the BatchWithdrawal
- This is the contract that implements `fa_withdrawal_precompile` and `fa_withdraw_precompile` interfaces.
- When user calls these precompile entrypoints, it make `N` copies of this call to the corresponding withdrawal precompile.
- Number of the calls `N` provided during origination.

In [8]:
bulk_withdrawal = BulkWithdrawalHelper.originate_from_file(
    web3=web3,
    account=etherlink_account,
    filename=make_filename('BulkWithdrawal'),
    constructor_args=(
        FA_WITHDRAWAL_PRECOMPILE,
        XTZ_WITHDRAWAL_PRECOMPILE,
        450,
    )
)
bulk_withdrawal.address

'0x1Cf25b150d52913AD102a32c5554358f1C6F4A7e'

In [2]:
bulk_withdrawal = BulkWithdrawalHelper.from_address(web3, etherlink_account, '0x1Cf25b150d52913AD102a32c5554358f1C6F4A7e')
bulk_withdrawal.address

'0x1Cf25b150d52913AD102a32c5554358f1C6F4A7e'

### FA Token deposit:
- It is required to deposit FA token to the contract so it would be able to execute withdrawals.

In [9]:
result = deposit.callback(
    token_bridge_helper_address=token_bridge_helper.address,
    amount=1_000,
    receiver_address=bulk_withdrawal.address,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL
)

Making deposit using Helper `KT1SSfdYB4JFTJoqVUe6yjVX9urLXiXge6e4`:
  - Executor: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/parisnet/`
  - Ticketer: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
  - Deposit params:
      * Smart Rollup address: `sr1G9N4WzRF79GeGKVd1vUDUg2NSrFGNGAfA`
      * Receiver address: `0x1Cf25b150d52913AD102a32c5554358f1C6F4A7e`
      * Amount: `1_000`
Successfully executed Deposit, tx hash: `oodxJDwp5S1MuRMmmDMDU4AfQdGfyrGMo2QHL1umpkHKdfZ8v9X`


### FA Token withdrawal:
- The withdraw CLI used with `batch_withdrawal.address` provided as withdrawal precompile contract.
- The `BatchWithdrawal` then executes `fa_withdrawal_precompile` multiple times spamming the same transaction.

In [5]:
withdraw.callback(
    erc20_proxy_address=erc20.address,
    tezos_side_router_address=ticketer.address,
    amount=1,
    ticketer_address_bytes=make_address_bytes(ticketer.address),
    ticket_content_bytes=ticketer.read_content().to_bytes_hex(),
    receiver_address=get_address(tezos_account),
    withdraw_precompile=bulk_withdrawal.address,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making FA withdrawal, ERC20 token: `0x03E39FF2b379FBcd9284Ab457113D82fF4daBBF4`:
  - Executor: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://etherlink.dipdup.net`
  - Withdrawal params:
      * Ticket owner: `0x03E39FF2b379FBcd9284Ab457113D82fF4daBBF4`
      * Receiver: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Router: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
      * Routing info: `0x0000d1b03118754a8e193b3e5d3a7cded976f4fd425601ce839f3e5187c0abec444760a58e950b675a334100`
      * Amount: `1`
      * Ticketer address bytes: `0x01ce839f3e5187c0abec444760a58e950b675a334100`
      * Content bytes: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54314b6639633575664c574538394e347a425775427173454d61354732674644525372070401000000046e616d650a00000022566172696f75732053656375726974792054657374696e67205363656e6172696f730704010000000673796d626f6c0a00000005765365635407040100000008746f6b656e5f69640a000000043930303007040

'0x6c6b6f03bfc10fd8f7fa056cb1646d9876aa5a41ca38fd301d9296a4e5d74d2a'

### Native deposit:
- To withdraw XTZ from the contract, it is required to deposit some.

In [6]:
opg_hash = xtz_deposit.callback(
    xtz_ticket_helper=XTZ_TICKET_HELPER,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    amount=1_000_000,
    receiver_address=bulk_withdrawal.address,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
)

Making XTZ deposit using Helper `KT1MJxf4KVN3sosR99VRG7WBbWTJtAyWUJt9`:
  - Executor: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/parisnet/`
  - XTZ deposit params:
      * Smart Rollup address: `sr1G9N4WzRF79GeGKVd1vUDUg2NSrFGNGAfA`
      * Receiver address: `0x1Cf25b150d52913AD102a32c5554358f1C6F4A7e`
      * Amount (mutez): `1_000_000`
Successfully executed XTZ deposit, tx hash: `oopRnZH6otLFksPRKnkPWKvymFjfkEv8GDjJqvYDd4xMPZA3mbu`


'oopRnZH6otLFksPRKnkPWKvymFjfkEv8GDjJqvYDd4xMPZA3mbu'

### Native withdrawal:
- The xtz withdraw CLI used with `batch_withdrawal.address` provided as withdrawal precompile contract.
- The `BatchWithdrawal` then executes `xtz_withdrawal_precompile` multiple times spamming the same transaction.

In [3]:
opg_hash = xtz_withdraw.callback(
    amount=10**12,
    xtz_withdraw_precompile=bulk_withdrawal.address,
    receiver_address='tz1burnburnburnburnburnburnburjAYjjX',
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL
)

Making XTZ withdrawal from `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`:
  - Executor: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://etherlink.dipdup.net`
  - Withdrawal params:
      * Receiver: `tz1burnburnburnburnburnburnburjAYjjX`
      * Amount (wei):: `1_000_000_000_000`
      * Amount (mutez):: `1`
Successfully initiated XTZ withdrawal, tx hash: `0x0c2779b51e96581f74cce9985106a85aa98bd12b0269233de94ae98e2826983f`


'0x0c2779b51e96581f74cce9985106a85aa98bd12b0269233de94ae98e2826983f'